In [75]:
import pandas as pd
import numpy as np


Data Extraction

In [76]:
#در این بخش دیتا ها را می خوانیم تا ساختار داده ها را بررسی کنیم
input_data=pd.read_csv("titanic.csv")
#input_data
input_data.info()
#input_data.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [77]:
#ابتدا چک می کنیم در کدام ستون ها مقدار خالی وجود دارد
input_data.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [78]:
#در ستون سن مقادیر خالی ای وجود دارد و برای این که در نتایج مان تاثیر نگذارد آن مقادیر را با میانگین سن مسافران پر می کنیم
input_data.fillna({"Age":input_data["Age"].mean()}, inplace=True)


In [79]:
#به این سه ستون نیازی نداریم چون هیچ ارتباطی بین آن ها وجود ندارد
input_data.drop(columns=["PassengerId","Name","Ticket"],inplace=True)


In [80]:
#با استفاده از این تابع همه داده ها را بررسی کرده و تا جایی که امکان دارد آن ها را کاهش حجم می دهیم
def auto_data_type(df):
    for col in df.select_dtypes(include=[np.number]).columns:
        min_value = df[col].min()
        max_value = df[col].max()

        if pd.api.types.is_float_dtype(df[col]):
            df[col] = df[col].astype(np.float32)
        elif min_value >= 0:
            if max_value <= 255:
                df[col] = df[col].astype(np.uint8)
            elif max_value <= 65535:
                df[col] = df[col].astype(np.uint16)
            elif max_value <= 4294967295:
                df[col] = df[col].astype(np.uint32)
            else:
                df[col] = df[col].astype(np.uint64)
        else:
            if -128 <= min_value and max_value <= 127:
                df[col] = df[col].astype(np.int8)
            elif -32768 <= min_value and max_value <= 32767:
                df[col] = df[col].astype(np.int16)
            elif -2147483648 <= min_value and max_value <= 2147483647:
                df[col] = df[col].astype(np.int32)
            else:
                df[col] = df[col].astype(np.int64)
    return df

clean_df=auto_data_type(input_data)
clean_df["Age"]=clean_df["Age"].astype(np.uint8)

#به بررسی مقدار حجم کاهش یا فته می پردازیم تا تغییرات را مشاهده کنیم. همان طور که دیده می شود حدود 43 درصد کاهش حجم رخ داده است.
clean_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    uint8  
 1   Pclass    891 non-null    uint8  
 2   Sex       891 non-null    object 
 3   Age       891 non-null    uint8  
 4   SibSp     891 non-null    uint8  
 5   Parch     891 non-null    uint8  
 6   Fare      891 non-null    float32
 7   Cabin     204 non-null    object 
 8   Embarked  889 non-null    object 
dtypes: float32(1), object(3), uint8(5)
memory usage: 28.8+ KB


In [81]:
#به دلیل آن که ستون های عددی برای نیاز داریم، ستون جنسیت را به صفر و یک تبدیل می کنیم
clean_df["Sex"] = clean_df["Sex"].map({"male":1, "female":2})
# هم چنین برای ستون Embarked هم در ابتدا به حرف اول جایی که سوار شدن نگاشت می کنیم
#clean_df["Embarked"] = clean_df["Embarked"].map({"S":1, "C":2, "Q":3})
# سپس مقادیر خالی را با استفاده از مد، پر می کنیم
#print(np.unique(clean_df["Embarked"].dropna(), return_counts=True) )
clean_df.fillna({"Embarked":"S"}, inplace=True)